## Evaluation of Weekly Performance based on MASE & SMAPE

In [14]:
!pip install utilsforecast -q

In [39]:
import pandas as pd
from utilsforecast.losses import smape, mase

In [40]:
merged_df=pd.read_excel("final_forecast_Daily_merged.xlsx")

In [41]:
train = pd.read_csv("Train_df_NueralForecast.csv")
forecast=merged_df.copy()
actuals=merged_df[["unique_id", "ds", "y"]]

In [42]:
train["ds"]=pd.to_datetime(train["ds"])
actuals["ds"]=pd.to_datetime(actuals["ds"])
forecast["ds"]=pd.to_datetime(forecast["ds"])

In [43]:
def evaluate(train, actuals, forecast, seasonality=365, id_col='unique_id', target_col='y'):
  fcst_and_actuals = pd.merge(actuals, forecast, on=['ds', 'unique_id'], how='left')
  models=[ "AutoARIMA", "AutoETS", "AutoTBATS", 'RandomForestRegressor','XGBRegressor', "LGBMRegressor", "TFT", "NHITS", "TimeGPT",'TimeGPT_finetuned', "TimesFM"]
  mase_val = mase(fcst_and_actuals, models=models, seasonality=seasonality, train_df=train, id_col='unique_id', target_col='y')
  mase_val = mase_val.mean(numeric_only=True)
  smape_val = smape(fcst_and_actuals, models=models,  id_col='unique_id', target_col='y')
  smape_val = smape_val.mean(numeric_only=True)
  results = pd.DataFrame({'mase': mase_val, 'smape': smape_val}, index=models)
  results.index.name = 'models'
  results = results.round(4)
  return results

In [44]:
daily_results=evaluate(train, actuals, forecast, seasonality=365, id_col='unique_id', target_col='y')

In [45]:
daily_results

,MASE,SMAPE
models,,
AutoARIMA,0.0981,0.1193
AutoETS,0.0969,0.1158
AutoTBATS,0.0942,0.1182
RandomForestRegressor,0.0961,0.1171
XGBRegressor,0.1010,0.1204
LGBMRegressor,0.0948,0.1165
TFT,0.0965,0.1177
NHITS,0.0989,0.1203
TimeGPT,0.1073,0.1264
